In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.chdir('/content/drive/MyDrive/0.리뷰 데이터 모음')

os.getcwd()

'/content/drive/MyDrive/0.리뷰 데이터 모음'

In [ ]:
import pandas as pd

review = pd.read_csv('labeling_total.csv')

review.head()

,Review,Label
0,매번 9시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱...,1.0
1,처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 ...,9.0
2,그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 ...,1.0
3,단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌...,0.0
4,ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘...,9.0


# 1. 데이터 전처리

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import re
import urllib.request

!pip3 install konlpy
from konlpy.tag import Okt

from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1) 정규 표현식 사용

In [ ]:
review['Review'] = review['Review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

review['Review'] = review['Review'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
review['Review'].replace('', np.nan, inplace = True)

review['Review'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


0     매번 시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱 ...
1     처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 ...
2     그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 ...
3     단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌...
4     ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘...
Name: Review, dtype: object

## 2) Empty 리뷰 제외

In [ ]:
review.loc[review.Review.isnull()].head()

,Review,Label
1488,NaN,1.0
1489,NaN,1.0
1933,NaN,9.0
1934,NaN,1.0
1935,NaN,2.0


In [ ]:
review = review.dropna(how = 'any')
review.reset_index(inplace = True , drop = True)

print(len(review.loc[review.Review.isnull()]))

0


# 2. 자카드 유사도

** 유사 리뷰 구별 방법 **

1.   (Stem , Norm) = (T , T) or (T , F) or (F , T) , (F , F)의 4 가지 방식으로 토큰화
2.   임의의 tokenized review에 대하여 나머지 tokenized review와 자카드 유사도 계산 
3.   임의의 tokenized review에 대하여 자카드 유사도가 0.6 이상인 리뷰를 list로 저장
4.   자카드 유사도가 0.6 이상인 리뷰의 존재 여부에 따라 0,1의 값을 갖는 Jaccard_YN (binary) 변수 생성
5.   감정별(긍정,부정,개선,기타) 전체 리뷰 중 Jaccard_YN == 1인 리뷰의 비율 확인
6.   Jaccard_YN == 1인 리뷰 중 감정별 분포 확인



## 1) (stem = True , norm = True)

In [ ]:
okt = Okt()

In [ ]:
review_token = []

for sentence in tqdm(review['Review']) :

    tokenized_sentence = okt.morphs(sentence, stem = True , norm = True) # 토큰화
    
    review_token.append(tokenized_sentence)

100%|██████████| 7327/7327 [00:44<00:00, 165.72it/s]


In [ ]:
print(review_token[:5])

[['매번', '시간', '넘다', '자다', '깊다', '잠들다', '못', '하고', '수시로', '깨다', '이', '거', '켜', '놨', '을', '때', '딱', '시간', '반', '자고', '알람', '을', '맞추다', '않다', '제때', '일어나다', '수', '있다', '좋다', '면서', '깨우다', '않다', '자다', '일어난다', '며', '차다', '좋아하다'], ['처음', '엔', '휴대폰', '에서', '나', '는', '소리', '가', '적응', '이', '되다', '않다', '머리', '가', '묵직하다', '불편하다', '데', '이틀', '지나다', '적응', '이', '끝나다', '건지다', '괜찮다', '지다'], ['그리고', '가끔', '아침', '에', '일어나다', '때', '머리', '가', '몽롱', '하다', '데', '그', '땐', '아침밥', '을', '먹다', '커피', '한잔', '을', '마시다', '주다', '차다', '개운하다', '하루', '를', '시작', '하다', '수', '있다'], ['단점', '가끔', '휴대폰', '소프트웨어', '가', '밤', '에', '자기', '마음대로', '업데이트', '되다', '그때', '폰', '이', '꺼지다', '켜다', '그', '날', '은', '지각', '하다', '거랍니', '다'], ['ㅜㅠ', '앱', '에', '표시', '되다', '광고', '에', '대한', '평가', '란', '이', '있다', '그렇다', '거', '신경', '안', '쓰다', '잠자다', '때', '만', '사용', '하다', '보다', '앱', '에', '광고', '가', '있다', '걸', '평가', '란', '을', '보고', '처음', '알다']]


In [ ]:
review_token_df = pd.DataFrame({'Token' : review_token})

review_token_df.head()

,Token
0,"[매번, 시간, 넘다, 자다, 깊다, 잠들다, 못, 하고, 수시로, 깨다, 이, 거..."
1,"[처음, 엔, 휴대폰, 에서, 나, 는, 소리, 가, 적응, 이, 되다, 않다, 머..."
2,"[그리고, 가끔, 아침, 에, 일어나다, 때, 머리, 가, 몽롱, 하다, 데, 그,..."
3,"[단점, 가끔, 휴대폰, 소프트웨어, 가, 밤, 에, 자기, 마음대로, 업데이트, ..."
4,"[ㅜㅠ, 앱, 에, 표시, 되다, 광고, 에, 대한, 평가, 란, 이, 있다, 그렇..."


In [ ]:
def jaccard_similarity(sentence_1, sentence_2) :

    s1 = set(sentence_1)
    s2 = set(sentence_2)

    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [ ]:
jaccard_idx = []

for i in tqdm(range(len(review_token_df))) :

  idx_list = []

  for j in range(len(review_token_df)) :

    if i != j and jaccard_similarity(review_token_df.iloc[i,0] , review_token_df.iloc[j,0]) > 0.6 :

      idx_list.append(j)

  jaccard_idx.append(idx_list)

100%|██████████| 7327/7327 [1:16:12<00:00,  1.60it/s]


### A. 전체 리뷰 중 유사 리뷰 비율

In [ ]:
review_copy = review.copy()

review_copy['Jaccard'] = jaccard_idx

review_copy['Jaccard_YN'] = list(np.repeat(1 , len(review_copy)))

for i in tqdm(range(len(review_copy))) :

  if len(review_copy.loc[i , 'Jaccard']) == 0 :

    review_copy.loc[i , 'Jaccard_YN'] = 0 

review_copy.head()

100%|██████████| 7327/7327 [00:02<00:00, 2696.46it/s]


,Review,Label,Jaccard,Jaccard_YN
0,매번 시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱 시...,1.0,[],0
1,처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 지...,9.0,[],0
2,그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 한...,1.0,[],0
3,단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌다...,0.0,[],0
4,ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘 ...,9.0,[],0


In [ ]:
con = review_copy['Jaccard_YN'] == 1
con_0 = review_copy['Label'] == 0
con_1 = review_copy['Label'] == 1
con_2 = review_copy['Label'] == 2
con_9 = review_copy['Label'] == 9

In [ ]:
print('전체 긍정 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_1 , :])/len(review_copy.loc[con_1 , :]))
print('전체 부정 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_0 , :])/len(review_copy.loc[con_0 , :]))
print('전체 개선 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_2 , :])/len(review_copy.loc[con_2 , :]))
print('전체 기타 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_9 , :])/len(review_copy.loc[con_9 , :]))

전체 긍정 리뷰 중 유사 리뷰의 비율 :  0.039777983348751156
전체 부정 리뷰 중 유사 리뷰의 비율 :  0.024279210925644917
전체 개선 리뷰 중 유사 리뷰의 비율 :  0.04554588077695914
전체 기타 리뷰 중 유사 리뷰의 비율 :  0.011400651465798045


### B. 유사 리뷰 중 감정별 리뷰 비율

In [ ]:
review_stem_T_norm_T = review.copy()
review_stem_T_norm_T['Jaccard'] = jaccard_idx

idx_list = []

for i in tqdm(range(len(review_stem_T_norm_T))) :

  if len(review_stem_T_norm_T.iloc[i,2]) > 0 :

    idx_list.append(i)

print('\n')
print(len(idx_list))

100%|██████████| 7327/7327 [00:00<00:00, 25655.88it/s]



252


In [ ]:
Stem_T_norm_T = review_stem_T_norm_T.iloc[idx_list , :]

In [ ]:
print('유사 리뷰 중 긍정 리뷰의 비율 : ' , len(review_stem_T_norm_T[review_stem_T_norm_T['Label'] == 1])/len(review_stem_T_norm_T))
print('유사 리뷰 중 부정 리뷰의 비율 : ' , len(review_stem_T_norm_T[review_stem_T_norm_T['Label'] == 0])/len(review_stem_T_norm_T))
print('유사 리뷰 중 개선 리뷰의 비율 : ' , len(review_stem_T_norm_T[review_stem_T_norm_T['Label'] == 2])/len(review_stem_T_norm_T))
print('유사 리뷰 중 기타 리뷰의 비율 : ' , len(review_stem_T_norm_T[review_stem_T_norm_T['Label'] == 9])/len(review_stem_T_norm_T))

유사 리뷰 중 긍정 리뷰의 비율 :  0.44260952640917156
유사 리뷰 중 부정 리뷰의 비율 :  0.269823938856285
유사 리뷰 중 개선 리뷰의 비율 :  0.20376688958646103
유사 리뷰 중 기타 리뷰의 비율 :  0.08379964514808244


## 2) (stem = False , norm = True)

In [ ]:
review_token = []

for sentence in tqdm(review['Review']) :

    tokenized_sentence = okt.morphs(sentence, stem = False , norm = True) # 토큰화
    
    review_token.append(tokenized_sentence)

100%|██████████| 7327/7327 [00:32<00:00, 226.28it/s]


In [ ]:
print(review_token[:5])

[['매번', '시간', '넘게', '자거나', '깊게', '잠들지', '못', '하고', '수시로', '깨는데', '이', '거', '켜', '놨', '을', '때', '딱', '시간', '반', '자고', '알람', '을', '맞추지', '않아도', '제때', '일어날', '수', '있어서', '좋았습니다', '면서', '깨우지', '않아도', '잘', '일어난다', '며', '참', '좋아하십니다'], ['처음', '엔', '휴대폰', '에서', '나', '는', '소리', '가', '적응', '이', '되지', '않아', '머리', '가', '묵직하고', '불편한', '데', '이틀', '지나면', '적응', '이', '끝난', '건지', '괜찮아', '집니다'], ['그리고', '가끔', '아침', '에', '일어났을', '때', '머리', '가', '몽롱', '한', '데', '그', '땐', '아침밥', '을', '먹거나', '커피', '한잔', '을', '마셔', '주면', '참', '개운한', '하루', '를', '시작', '할', '수', '있습니다'], ['단점', '가끔', '휴대폰', '소프트웨어', '가', '밤', '에', '자기', '마음대로', '업데이트', '되는데', '그때', '폰', '이', '꺼졌다', '켜지면', '그', '날', '은', '지각', '하는', '거랍니', '다'], ['ㅜㅠ', '앱', '에', '표시', '되는', '광고', '에', '대한', '평가', '란', '이', '있었는데', '그런', '거', '신경', '안', '쓰고', '잠잘', '때', '만', '사용', '하다', '보니', '앱', '에', '광고', '가', '있었다는', '걸', '평가', '란', '을', '보고', '처음', '알았습니다']]


In [ ]:
review_token_df = pd.DataFrame({'Token' : review_token})

review_token_df.head()

,Token
0,"[매번, 시간, 넘게, 자거나, 깊게, 잠들지, 못, 하고, 수시로, 깨는데, 이,..."
1,"[처음, 엔, 휴대폰, 에서, 나, 는, 소리, 가, 적응, 이, 되지, 않아, 머..."
2,"[그리고, 가끔, 아침, 에, 일어났을, 때, 머리, 가, 몽롱, 한, 데, 그, ..."
3,"[단점, 가끔, 휴대폰, 소프트웨어, 가, 밤, 에, 자기, 마음대로, 업데이트, ..."
4,"[ㅜㅠ, 앱, 에, 표시, 되는, 광고, 에, 대한, 평가, 란, 이, 있었는데, ..."


In [ ]:
jaccard_idx = []

for i in tqdm(range(len(review_token_df))) :

  idx_list = []

  for j in range(len(review_token_df)) :

    if i != j and jaccard_similarity(review_token_df.iloc[i,0] , review_token_df.iloc[j,0]) > 0.6 :

      idx_list.append(j)

  jaccard_idx.append(idx_list)

100%|██████████| 7327/7327 [1:16:14<00:00,  1.60it/s]


### A. 전체 리뷰 중 유사 리뷰 비율

In [ ]:
review_copy = review.copy()

review_copy['Jaccard'] = jaccard_idx

review_copy['Jaccard_YN'] = list(np.repeat(1 , len(review_copy)))

for i in tqdm(range(len(review_copy))) :

  if len(review_copy.loc[i , 'Jaccard']) == 0 :

    review_copy.loc[i , 'Jaccard_YN'] = 0 

review_copy.head()

100%|██████████| 7327/7327 [00:02<00:00, 2636.61it/s]


,Review,Label,Jaccard,Jaccard_YN
0,매번 시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱 시...,1.0,[],0
1,처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 지...,9.0,[],0
2,그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 한...,1.0,[],0
3,단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌다...,0.0,[],0
4,ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘 ...,9.0,[],0


In [ ]:
con = review_copy['Jaccard_YN'] == 1
con_0 = review_copy['Label'] == 0
con_1 = review_copy['Label'] == 1
con_2 = review_copy['Label'] == 2
con_9 = review_copy['Label'] == 9

In [ ]:
print('전체 긍정 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_1 , :])/len(review_copy.loc[con_1 , :]))
print('전체 부정 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_0 , :])/len(review_copy.loc[con_0 , :]))
print('전체 개선 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_2 , :])/len(review_copy.loc[con_2 , :]))
print('전체 기타 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_9 , :])/len(review_copy.loc[con_9 , :]))

전체 긍정 리뷰 중 유사 리뷰의 비율 :  0.027752081406105456
전체 부정 리뷰 중 유사 리뷰의 비율 :  0.013657056145675266
전체 개선 리뷰 중 유사 리뷰의 비율 :  0.03148024112525117
전체 기타 리뷰 중 유사 리뷰의 비율 :  0.006514657980456026


### B. 유사 리뷰 중 감정별 리뷰 비율

In [ ]:
review_stem_F_norm_T = review.copy()
review_stem_F_norm_T['Jaccard'] = jaccard_idx

idx_list = []

for i in tqdm(range(len(review_stem_F_norm_T))) :

  if len(review_stem_F_norm_T.iloc[i,2]) > 0 :

    idx_list.append(i)

print('\n')
print(len(idx_list))

100%|██████████| 7327/7327 [00:00<00:00, 26465.64it/s]



168


In [ ]:
Stem_F_norm_T = review_stem_F_norm_T.iloc[idx_list , :]

In [ ]:
print('유사 리뷰 중 긍정 리뷰의 비율 : ' , len(Stem_F_norm_T[Stem_F_norm_T['Label'] == 1])/len(Stem_F_norm_T))
print('유사 리뷰 중 부정 리뷰의 비율 : ' , len(Stem_F_norm_T[Stem_F_norm_T['Label'] == 0])/len(Stem_F_norm_T))
print('유사 리뷰 중 개선 리뷰의 비율 : ' , len(Stem_F_norm_T[Stem_F_norm_T['Label'] == 2])/len(Stem_F_norm_T))
print('유사 리뷰 중 기타 리뷰의 비율 : ' , len(Stem_F_norm_T[Stem_F_norm_T['Label'] == 9])/len(Stem_F_norm_T))

유사 리뷰 중 긍정 리뷰의 비율 :  0.5357142857142857
유사 리뷰 중 부정 리뷰의 비율 :  0.16071428571428573
유사 리뷰 중 개선 리뷰의 비율 :  0.27976190476190477
유사 리뷰 중 기타 리뷰의 비율 :  0.023809523809523808


## 3) (stem = True , norm = False)

In [ ]:
review_token = []

for sentence in tqdm(review['Review']) :

    tokenized_sentence = okt.morphs(sentence, stem = True , norm = False) # 토큰화
    
    review_token.append(tokenized_sentence)

100%|██████████| 7336/7336 [00:18<00:00, 387.59it/s]


In [ ]:
print(review_token[:5])

[['매번', '시간', '넘다', '자다', '깊다', '잠들다', '못', '하고', '수시로', '깨다', '이', '거', '켜', '놨', '을', '때', '딱', '시간', '반', '자고', '알람', '을', '맞추다', '않다', '제때', '일어나다', '수', '있다', '좋다', '면서', '깨우다', '않다', '자다', '일어난다', '며', '차다', '좋아하다'], ['처음', '엔', '휴대폰', '에서', '나', '는', '소리', '가', '적응', '이', '되다', '않다', '머리', '가', '묵직하다', '불편하다', '데', '이틀', '지나다', '적응', '이', '끝나다', '건지다', '괜찮다', '지다'], ['그리고', '가끔', '아침', '에', '일어나다', '때', '머리', '가', '몽롱', '하다', '데', '그', '땐', '아침밥', '을', '먹다', '커피', '한잔', '을', '마시다', '주다', '차다', '개운하다', '하루', '를', '시작', '하다', '수', '있다'], ['단점', '가끔', '휴대폰', '소프트웨어', '가', '밤', '에', '자기', '마음대로', '업데이트', '되다', '그때', '폰', '이', '꺼지다', '켜다', '그', '날', '은', '지각', '하다', '거랍니', '다'], ['ㅜㅠ', '앱', '에', '표시', '되다', '광고', '에', '대한', '평가', '란', '이', '있다', '그렇다', '거', '신경', '안', '쓰다', '잠자다', '때', '만', '사용', '하다', '보다', '앱', '에', '광고', '가', '있다', '걸', '평가', '란', '을', '보고', '처음', '알다']]


In [ ]:
review_token_df = pd.DataFrame({'Token' : review_token})

review_token_df.head()

,Token
0,"[매번, 9시간, 넘다, 자다, 깊다, 잠들다, 못, 하고, 수시로, 깨다, 이, ..."
1,"[처음, 엔, 휴대폰, 에서, 나, 는, 소리, 가, 적응, 이, 되다, 않다, 머..."
2,"[그리고, 가끔, 아침, 에, 일어나다, 때, 머리, 가, 몽롱, 하다, 데, 그,..."
3,"[단점, 가끔, 휴대폰, 소프트웨어, 가, 밤, 에, 자기, 마음대로, 업데이트, ..."
4,"[ㅜㅠ, 앱, 에, 표시, 되다, 광고, 에, 대한, 평가, 란, 이, 있다, 그렇..."


In [ ]:
jaccard_idx = []

for i in tqdm(range(len(review_token_df))) :

  idx_list = []

  for j in range(len(review_token_df)) :

    if i != j and jaccard_similarity(review_token_df.iloc[i,0] , review_token_df.iloc[j,0]) > 0.6 :

      idx_list.append(j)

  jaccard_idx.append(idx_list)

100%|██████████| 7336/7336 [1:16:59<00:00,  1.59it/s]


### A. 전체 리뷰 중 유사 리뷰 비율

In [ ]:
review_copy = review.copy()

review_copy['Jaccard'] = jaccard_idx

review_copy['Jaccard_YN'] = list(np.repeat(1 , len(review_copy)))

for i in tqdm(range(len(review_copy))) :

  if len(review_copy.loc[i , 'Jaccard']) == 0 :

    review_copy.loc[i , 'Jaccard_YN'] = 0 

review_copy.head()

100%|██████████| 7336/7336 [00:02<00:00, 2581.59it/s]


,Review,Label,Jaccard,Jaccard_YN
0,매번 9시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱...,1.0,[],0
1,처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 ...,9.0,[],0
2,그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 ...,1.0,[],0
3,단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌...,0.0,[],0
4,ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘...,9.0,[],0


In [ ]:
con = review_copy['Jaccard_YN'] == 1
con_0 = review_copy['Label'] == 0
con_1 = review_copy['Label'] == 1
con_2 = review_copy['Label'] == 2
con_9 = review_copy['Label'] == 9

In [ ]:
print('전체 긍정 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_1 , :])/len(review_copy.loc[con_1 , :]))
print('전체 부정 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_0 , :])/len(review_copy.loc[con_0 , :]))
print('전체 개선 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_2 , :])/len(review_copy.loc[con_2 , :]))
print('전체 기타 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_9 , :])/len(review_copy.loc[con_9 , :]))

전체 긍정 리뷰 중 유사 리뷰의 비율 :  0.03970452446906741
전체 부정 리뷰 중 유사 리뷰의 비율 :  0.02276176024279211
전체 개선 리뷰 중 유사 리뷰의 비율 :  0.04548494983277592
전체 기타 리뷰 중 유사 리뷰의 비율 :  0.00975609756097561


### B. 유사 리뷰 중 감정별 리뷰 비율

In [ ]:
review_stem_T_norm_F = review.copy()
review_stem_T_norm_F['Jaccard'] = jaccard_idx

idx_list = []

for i in tqdm(range(len(review_stem_T_norm_F))) :

  if len(review_stem_T_norm_F.iloc[i,2]) > 0 :

    idx_list.append(i)

print('\n')
print(len(idx_list))

100%|██████████| 7336/7336 [00:00<00:00, 26257.46it/s]



248


In [ ]:
Stem_T_norm_F = review_stem_T_norm_F.iloc[idx_list , :]

In [ ]:
print('유사 리뷰 중 긍정 리뷰의 비율 : ' , len(Stem_T_norm_F[Stem_T_norm_F['Label'] == 1])/len(Stem_T_norm_F))
print('유사 리뷰 중 부정 리뷰의 비율 : ' , len(Stem_T_norm_F[Stem_T_norm_F['Label'] == 0])/len(Stem_T_norm_F))
print('유사 리뷰 중 개선 리뷰의 비율 : ' , len(Stem_T_norm_F[Stem_T_norm_F['Label'] == 2])/len(Stem_T_norm_F))
print('유사 리뷰 중 기타 리뷰의 비율 : ' , len(Stem_T_norm_F[Stem_T_norm_F['Label'] == 9])/len(Stem_T_norm_F))

유사 리뷰 중 긍정 리뷰의 비율 :  0.5201612903225806
유사 리뷰 중 부정 리뷰의 비율 :  0.1814516129032258
유사 리뷰 중 개선 리뷰의 비율 :  0.27419354838709675
유사 리뷰 중 기타 리뷰의 비율 :  0.024193548387096774


## 4) (stem = False , norm = False)

In [ ]:
review_token = []

for sentence in tqdm(review['Review']) :

    tokenized_sentence = okt.morphs(sentence, stem = False , norm = False) # 토큰화
    
    review_token.append(tokenized_sentence)

100%|██████████| 7336/7336 [00:18<00:00, 390.93it/s]


In [ ]:
print(review_token[:5])

[['매번', '시간', '넘게', '자거나', '깊게', '잠들지', '못', '하고', '수시로', '깨는데', '이', '거', '켜', '놨', '을', '때', '딱', '시간', '반', '자고', '알람', '을', '맞추지', '않아도', '제때', '일어날', '수', '있어서', '좋았습니다', '면서', '깨우지', '않아도', '잘', '일어난다', '며', '참', '좋아하십니다'], ['처음', '엔', '휴대폰', '에서', '나', '는', '소리', '가', '적응', '이', '되지', '않아', '머리', '가', '묵직하고', '불편한', '데', '이틀', '지나면', '적응', '이', '끝난', '건지', '괜찮아', '집니다'], ['그리고', '가끔', '아침', '에', '일어났을', '때', '머리', '가', '몽롱', '한', '데', '그', '땐', '아침밥', '을', '먹거나', '커피', '한잔', '을', '마셔', '주면', '참', '개운한', '하루', '를', '시작', '할', '수', '있습니다'], ['단점', '가끔', '휴대폰', '소프트웨어', '가', '밤', '에', '자기', '마음대로', '업데이트', '되는데', '그때', '폰', '이', '꺼졌다', '켜지면', '그', '날', '은', '지각', '하는', '거랍니', '다'], ['ㅜㅠ', '앱', '에', '표시', '되는', '광고', '에', '대한', '평가', '란', '이', '있었는데', '그런', '거', '신경', '안', '쓰고', '잠잘', '때', '만', '사용', '하다', '보니', '앱', '에', '광고', '가', '있었다는', '걸', '평가', '란', '을', '보고', '처음', '알았습니다']]


In [ ]:
review_token_df = pd.DataFrame({'Token' : review_token})

review_token_df.head()

,Token
0,"[매번, 9시간, 넘게, 자거나, 깊게, 잠들지, 못, 하고, 수시로, 깨는데, 이..."
1,"[처음, 엔, 휴대폰, 에서, 나, 는, 소리, 가, 적응, 이, 되지, 않아, 머..."
2,"[그리고, 가끔, 아침, 에, 일어났을, 때, 머리, 가, 몽롱, 한, 데, 그, ..."
3,"[단점, 가끔, 휴대폰, 소프트웨어, 가, 밤, 에, 자기, 마음대로, 업데이트, ..."
4,"[ㅜㅠ, 앱, 에, 표시, 되는, 광고, 에, 대한, 평가, 란, 이, 있었는데, ..."


In [ ]:
jaccard_idx = []

for i in tqdm(range(len(review_token_df))) :

  idx_list = []

  for j in range(len(review_token_df)) :

    if i != j and jaccard_similarity(review_token_df.iloc[i,0] , review_token_df.iloc[j,0]) > 0.6 :

      idx_list.append(j)

  jaccard_idx.append(idx_list)

100%|██████████| 7336/7336 [1:17:01<00:00,  1.59it/s]


### A. 전체 리뷰 중 유사 리뷰 비율

In [ ]:
review_copy = review.copy()

review_copy['Jaccard'] = jaccard_idx

review_copy['Jaccard_YN'] = list(np.repeat(1 , len(review_copy)))

for i in tqdm(range(len(review_copy))) :

  if len(review_copy.loc[i , 'Jaccard']) == 0 :

    review_copy.loc[i , 'Jaccard_YN'] = 0 

review_copy.head()

100%|██████████| 7336/7336 [00:02<00:00, 2589.65it/s]


,Review,Label,Jaccard,Jaccard_YN
0,매번 9시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱...,1.0,[],0
1,처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 ...,9.0,[],0
2,그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 ...,1.0,[],0
3,단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌...,0.0,[],0
4,ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘...,9.0,[],0


In [ ]:
con = review_copy['Jaccard_YN'] == 1
con_0 = review_copy['Label'] == 0
con_1 = review_copy['Label'] == 1
con_2 = review_copy['Label'] == 2
con_9 = review_copy['Label'] == 9

In [ ]:
print('전체 긍정 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_1 , :])/len(review_copy.loc[con_1 , :]))
print('전체 부정 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_0 , :])/len(review_copy.loc[con_0 , :]))
print('전체 개선 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_2 , :])/len(review_copy.loc[con_2 , :]))
print('전체 기타 리뷰 중 유사 리뷰의 비율 : ' , len(review_copy.loc[con & con_9 , :])/len(review_copy.loc[con_9 , :]))

전체 긍정 리뷰 중 유사 리뷰의 비율 :  0.027700831024930747
전체 부정 리뷰 중 유사 리뷰의 비율 :  0.012645422357106728
전체 개선 리뷰 중 유사 리뷰의 비율 :  0.031438127090301006
전체 기타 리뷰 중 유사 리뷰의 비율 :  0.0065040650406504065


### B. 유사 리뷰 중 감정별 리뷰 비율

In [ ]:
review_stem_F_norm_F = review.copy()
review_stem_F_norm_F['Jaccard'] = jaccard_idx

idx_list = []

for i in tqdm(range(len(review_stem_F_norm_F))) :

  if len(review_stem_F_norm_F.iloc[i,2]) > 0 :

    idx_list.append(i)

print('\n')
print(len(idx_list))

100%|██████████| 7336/7336 [00:00<00:00, 23458.73it/s]



166


In [ ]:
Stem_F_norm_F = review_stem_F_norm_F.iloc[idx_list , :]

In [ ]:
print('유사 리뷰 중 긍정 리뷰의 비율 : ' , len(Stem_F_norm_F[Stem_F_norm_F['Label'] == 1])/len(Stem_F_norm_F))
print('유사 리뷰 중 부정 리뷰의 비율 : ' , len(Stem_F_norm_F[Stem_F_norm_F['Label'] == 0])/len(Stem_F_norm_F))
print('유사 리뷰 중 개선 리뷰의 비율 : ' , len(Stem_F_norm_F[Stem_F_norm_F['Label'] == 2])/len(Stem_F_norm_F))
print('유사 리뷰 중 기타 리뷰의 비율 : ' , len(Stem_F_norm_F[Stem_F_norm_F['Label'] == 9])/len(Stem_F_norm_F))

유사 리뷰 중 긍정 리뷰의 비율 :  0.5421686746987951
유사 리뷰 중 부정 리뷰의 비율 :  0.15060240963855423
유사 리뷰 중 개선 리뷰의 비율 :  0.28313253012048195
유사 리뷰 중 기타 리뷰의 비율 :  0.024096385542168676
